In [33]:
import pandas as pd
import scanpy as sc
import numpy as np

In [22]:
# .obs['predicted.celltype'] - seurat
# .obs['CoDi'] - CoDi
# .obs['CoDi_dist'] - CoDi_dist
# .obs['sample'] - sample

adata = sc.read_h5ad('../data/merged.h5ad')
adata

AnnData object with n_obs × n_vars = 34078 × 20388
    obs: 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.0.3', 'seurat_clusters', 'predicted.id', 'prediction.score.CD14..monocyte', 'prediction.score.Dendritic.cell', 'prediction.score.Cytotoxic.T.cell', 'prediction.score.Plasmacytoid.dendritic.cell', 'prediction.score.B.cell', 'prediction.score.Natural.killer.cell', 'prediction.score.CD4..T.cell', 'prediction.score.max', 'predicted.celltype.score', 'predicted.celltype', 'CoDi', 'CoDi_confidence', 'CoDi_dist', 'CoDi_confidence_dist', 'CoDi_contrastive', 'CoDi_confidence_contrastive', 'sample'
    obsm: 'X_pca', 'X_ref.pca', 'X_ref.umap', 'X_umap'
    layers: 'counts'

In [74]:
adata_ref = sc.read_h5ad('../data/ref_broad_pbmc/sc.h5ad')
adata_ref.var.set_index('var.features', inplace=True)
adata_ref

AnnData object with n_obs × n_vars = 28055 × 33694
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nGene', 'nUMI', 'percent.mito', 'Cluster', 'CellType', 'Experiment', 'Method'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [45]:
samples = sorted(adata.obs['sample'].unique())
# Normalize and log1p per sample
processed_adata_list = []
for sample in samples:
    ad_sample = adata[adata.obs['sample'] == sample].copy()
    sc.pp.normalize_total(ad_sample, target_sum=1e4)
    sc.pp.log1p(ad_sample)
    processed_adata_list.append(ad_sample)

# Concatenate normalized samples back
adata_norm = processed_adata_list

In [121]:
# Define marker genes sets
marker_sets = {
    "B cell": [
        "BANK1", "BACH2", "CD19", "CD22", "CD24", "CD37", "CD74",
        "CD79A", "CD79B", "CR2", "FCER2", "HLA-DRA", "IGHD", "IGHM",
        "MS4A1", "PAX5", "SPIB", "TNFRSF13B", "TNFRSF13C"
    ],
    "CD14+ monocyte": [
        "CD14", "LYZ", "S100A8", "S100A9", "FCN1", "CCR2", "CSF3R",
        "FCGR3A", "ITGAM", "CX3CR1", "LST1", "LGALS3", "CTSS",
        "TREM1", "IL1B", "IFI27", "TNF", "ICAM1", "TLR4"
    ],
    "CD4+ T cell": [
        "CD3D", "CD3E", "CD3G", "CD4", "IL7R", "CCR7", "LTB", "SELL",
        "TCF7", "LEF1", "TRAC", "TRBC1", "TRBC2"
    ],
    "Cytotoxic T cell": [
        "CD8A", "CD8B", "GZMA", "GZMB", "GZMK", "PRF1", "GNLY",
        "NKG7", "KLRD1", "CTSW", "TRAC", "CD2", "CD27", "IFNG"
    ],
    "Natural killer cell": [
        "NCAM1", "NKG7", "GNLY", "KLRD1", "KLRF1", "GZMA", "GZMB",
        "GZMK", "PRF1", "FCGR3A", "FGFBP2", "KLRC1", "NCR1",
        "NCR3", "XCL1", "XCL2", "TBX21", "ZBTB16"
    ]
}
for k, v in marker_sets.items():
    print(len(v))

all_genes = set(adata.var.index).intersection(set(adata_ref.var.index))
marker_sets = {
    cell_type: [gene for gene in genes if gene in adata.var.index]
    for cell_type, genes in marker_sets.items()
}
print('-' * 50)
for k, v in marker_sets.items():
    print(len(v))
marker_sets

19
19
13
14
18
--------------------------------------------------
16
19
10
13
18


{'B cell': ['BANK1',
  'BACH2',
  'CD19',
  'CD22',
  'CD37',
  'CD74',
  'CD79A',
  'CD79B',
  'CR2',
  'FCER2',
  'HLA-DRA',
  'MS4A1',
  'PAX5',
  'SPIB',
  'TNFRSF13B',
  'TNFRSF13C'],
 'CD14+ monocyte': ['CD14',
  'LYZ',
  'S100A8',
  'S100A9',
  'FCN1',
  'CCR2',
  'CSF3R',
  'FCGR3A',
  'ITGAM',
  'CX3CR1',
  'LST1',
  'LGALS3',
  'CTSS',
  'TREM1',
  'IL1B',
  'IFI27',
  'TNF',
  'ICAM1',
  'TLR4'],
 'CD4+ T cell': ['CD3D',
  'CD3E',
  'CD3G',
  'CD4',
  'IL7R',
  'CCR7',
  'LTB',
  'SELL',
  'TCF7',
  'LEF1'],
 'Cytotoxic T cell': ['CD8A',
  'CD8B',
  'GZMA',
  'GZMB',
  'GZMK',
  'PRF1',
  'GNLY',
  'NKG7',
  'KLRD1',
  'CTSW',
  'CD2',
  'CD27',
  'IFNG'],
 'Natural killer cell': ['NCAM1',
  'NKG7',
  'GNLY',
  'KLRD1',
  'KLRF1',
  'GZMA',
  'GZMB',
  'GZMK',
  'PRF1',
  'FCGR3A',
  'FGFBP2',
  'KLRC1',
  'NCR1',
  'NCR3',
  'XCL1',
  'XCL2',
  'TBX21',
  'ZBTB16']}

In [150]:
# with open("output.txt", "w") as f:
#     for key in marker_sets:
#         value_list = marker_sets[key]
#         # Ensure each value is a plain string and not quoted
#         value_str = ", ".join(v.replace('"', '').replace("'", "") for v in value_list)
#         f.write(f"{key}: {value_str}\n")

In [96]:
# Calculate marker genes for all annotations and for all cell types
annotation_columns = ['predicted.celltype', 'CoDi', 'CoDi_dist']

results = []

for ind, sample in enumerate(samples):
    ad_sample = adata_norm[ind].copy()
    print(f"Processing sample: {sample}")

    for annot_col in annotation_columns:
        if annot_col not in ad_sample.obs:
            print(f"Warning: {annot_col} not found in obs")
            continue

        for cell_type in marker_sets.keys():
            # Check if enough cells of this type in the annotation
            mask = ad_sample.obs[annot_col] == cell_type
            if mask.sum() < 10:
                print(mask.sum() )
                continue

            # Run rank genes groups to find markers for this cell type vs others
            sc.tl.rank_genes_groups(ad_sample,
                                    groupby=annot_col,
                                    groups=[cell_type],
                                    reference='rest',
                                    method='wilcoxon',
                                    n_genes=100,
                                    use_raw=False)

            marker_genes = ad_sample.uns['rank_genes_groups']['names'][cell_type]
            known_markers = set(marker_sets[cell_type])
            found_markers = set(marker_genes).intersection(known_markers)
            score = np.round(len(found_markers) / len(known_markers), 2)

            results.append({
                'sample': sample,
                'annotation': annot_col,
                'cell_type': cell_type,
                'score': score,
                'found_markers': list(found_markers),
                'top_markers': list(marker_genes[:10])
            })

# Convert results to DataFrame
df_results = pd.DataFrame(results)

Processing sample: sample_1
Calc rank sum


/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Calc rank sum


/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Calc rank sum


...

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Calc rank sum


/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Calc rank sum


/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [95]:
# Caclulate marker genes in reference sample
annotation_columns = ['CellType']

results_ref = []


ad_sample = adata_ref

for annot_col in annotation_columns:
    if annot_col not in ad_sample.obs:
        print(f"Warning: {annot_col} not found in obs")
        continue

    for cell_type in marker_sets.keys():
        # Check if enough cells of this type in the annotation
        mask = ad_sample.obs[annot_col] == cell_type
        if mask.sum() < 10:
            print(mask.sum() )
            continue

        # Run rank genes groups to find markers for this cell type vs others
        sc.tl.rank_genes_groups(ad_sample,
                                groupby=annot_col,
                                groups=[cell_type],
                                reference='rest',
                                method='wilcoxon',
                                n_genes=100,
                                use_raw=False)

        marker_genes = ad_sample.uns['rank_genes_groups']['names'][cell_type]
        known_markers = set(marker_sets[cell_type])
        found_markers = set(marker_genes).intersection(known_markers)
        score = np.round(len(found_markers) / len(known_markers), 2)

        results_ref.append({
            'sample': sample,
            'annotation': annot_col,
            'cell_type': cell_type,
            'score': score,
            'found_markers': list(found_markers),
            'top_markers': list(marker_genes[:10])
        })
        display(score)

# Convert results to DataFrame
df_results_ref = pd.DataFrame(results_ref)
df_results_ref
# df_results.to_csv('ann_results.csv', index=False)

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0.44

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0.42

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


...

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0.69

/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0.5

,sample,annotation,cell_type,score,found_markers,top_markers
0,sample_4,CellType,B cell,0.44,"[MS4A1, CD79A, BANK1, HLA-DRA, CD37, CD74, CD79B]","[CD74, HLA-DRA, CD79A, MS4A1, IGHM, HLA-DRB1, ..."
1,sample_4,CellType,CD14+ monocyte,0.42,"[CD14, CSF3R, FCN1, S100A9, S100A8, LST1, LYZ,...","[LYZ, S100A9, S100A8, FCN1, FOS, CST3, PSAP, T..."
2,sample_4,CellType,CD4+ T cell,0.20,"[IL7R, LTB]","[IL7R, nan, LTB, nan, TRAC, nan, IL32, nan, na..."
3,sample_4,CellType,Cytotoxic T cell,0.69,"[CTSW, GZMB, CD8B, GZMA, KLRD1, NKG7, PRF1, CD...","[CCL5, NKG7, CST7, GZMH, IL32, CTSW, GZMA, FGF..."
4,sample_4,CellType,Natural killer cell,0.50,"[NKG7, GZMB, KLRF1, KLRD1, FCGR3A, PRF1, GZMA,...","[GNLY, NKG7, PRF1, GZMB, GZMA, KLRD1, KLRF1, C..."


In [107]:
res_grupped = pd.concat([df_results, df_results_ref]).groupby(['annotation', 'cell_type'])['score'].mean()

In [112]:
res_grupped.reset_index()

,annotation,cell_type,score
0,CellType,B cell,0.4400
1,CellType,CD14+ monocyte,0.4200
2,CellType,CD4+ T cell,0.2000
3,CellType,Cytotoxic T cell,0.6900
4,CellType,Natural killer cell,0.5000
5,CoDi,B cell,0.6200
6,CoDi,CD14+ monocyte,0.1225
7,CoDi,CD4+ T cell,0.4500
8,CoDi,Cytotoxic T cell,0.4025
9,CoDi,Natural killer cell,0.3600


In [149]:
# Assuming df is your original DataFrame
pivoted_df = res_grupped.reset_index().pivot(index='annotation', columns='cell_type')
pivoted_df.loc[:, 'Mean'] = pivoted_df.apply(lambda x: np.mean(x), axis=1)
pivoted_df = pivoted_df.reset_index()
pivoted_df['annotation'] = pivoted_df['annotation'].apply(lambda x: x.replace('CellType', 'scRNA reference').replace('predicted.celltype', 'Seurat'))
pivoted_df = pivoted_df.set_index('annotation')
pivoted_df.columns = pivoted_df.columns.get_level_values(1)
pivoted_df = pivoted_df.map(lambda x: np.round(x, 4))
pivoted_df.to_csv('ann_results.csv')
pivoted_df

cell_type,B cell,CD14+ monocyte,CD4+ T cell,Cytotoxic T cell,Natural killer cell,
annotation,,,,,,
scRNA reference,0.44,0.4200,0.20,0.6900,0.5000,0.4500
CoDi,0.62,0.1225,0.45,0.4025,0.3600,0.3910
CoDi_dist,0.62,0.1225,0.45,0.4400,0.4275,0.4120
Seurat,0.62,0.1225,0.45,0.4200,NaN,0.4031
